**Objetivo:** Leer y estandarizar los datasets (SIES, DEMRE, IVM, georreferenciación, etc.)

- Cargar datos de todas las fuentes
- Verificar formatos y columnas clave
- Establecer claves de unión (`MRUN`, `RBD`, códigos de institución)
- Guardar versiones limpias en `data/processed/` (opcional)

In [ ]:
import utils
from pathlib import Path

In [ ]:
# Lectura 
path = "../data/raw/2021"
list( Path(path).glob('*'))

In [ ]:
# Ruta base
path = '../data/raw'
path_base = Path(path)
year = 2021

# Obtener paths del año
path_year = utils.obtener_paths_anio(path_base, year)
path_year

In [ ]:
# Leer conjuntos individuales
set_a = utils.leer_conjunto_a_desde_path(path_year['matricula'], utils.region_dict)
set_b = utils.leer_conjunto_b_desde_path(path_year['puntajes'], utils.region_dict)
set_c = utils.leer_conjunto_c_desde_path(path_year['establecimientos'])
set_d = utils.leer_conjunto_d_desde_path(path_year['inmuebles_ies'], utils.region_dict)
set_e = utils.leer_conjunto_e_desde_path(path_year['ivm'])

In [ ]:
# Filtrar A1
resultados_a = utils.generar_conjuntos_filtrados(set_a, year)
set_a0 = resultados_a['A0']
set_aa = resultados_a['A']
set_a1 = resultados_a['A1']


In [ ]:
# Carpeta de salida
output_path = Path(f'../data/clean/{year}')
output_path.mkdir(parents=True, exist_ok=True)

# Guardar conjuntos individuales
set_a.to_csv(output_path / 'set_a.csv', index=False)
set_a0.to_csv(output_path / 'set_a0.csv', index=False)
set_aa.to_csv(output_path / 'set_aa.csv', index=False)
set_a1.to_csv(output_path / 'set_a1.csv', index=False)
set_b.to_csv(output_path / 'set_b.csv', index=False)
set_c.to_csv(output_path / 'set_c.csv', index=False)
set_d.to_csv(output_path / 'set_d.csv', index=False)
set_e.to_csv(output_path / 'set_e.csv', index=False)

In [ ]:
# Generar y guardar conjuntos integrados
utils.generar_conjuntos_abcde(
    set_a=set_a1,
    set_b=set_b,
    set_c=set_c,
    set_d=set_d,
    set_e=set_e,
    output_path=output_path
)

**Objetivo:** Explorar y analizar los datos limpios generados en la etapa de lectura.

- Revisar estructura general y dimensiones de los datasets  
- Analizar variables clave y su distribución  
- Identificar outliers, datos faltantes y relaciones relevantes  
- Generar visualizaciones exploratorias para apoyar el análisis  


In [1]:
import plots
import os
import pandas as pd
from pathlib import Path

In [2]:
# Lectura de datos

year = 2021
DATA_DIR = Path(f"../data/clean/{year}")

# Buscar archivos
archivos = list(DATA_DIR.glob('set_*.csv'))

# Leer archivos y guardar en un diccionario
data = {
    archivo.stem: pd.read_csv(archivo)
    for archivo in archivos
}

In [3]:
df_porcentaje = plots.preparar_dataset_tipo_universidad(data['set_a'], plots.tipos_universidad, plots.orden_regiones)
plots.plotly_tipo_universidad_por_region(df_porcentaje)

In [4]:
df_porcentaje = plots.preparar_dataset_tipodepen(data['set_c'], plots.regiones_dict, plots.tipodepen_dict, plots.index_order)
plots.plotly_tipodepen_por_region(df_porcentaje)

In [5]:
df_ivm = plots.preparar_dataset_ivm(data['set_abcde'], plots.regiones_dict, plots.index_order)
plots.plotly_ivm_por_region(df_ivm)

In [6]:

matriz = plots.preparar_matriz_movilidad(data['set_ab'], plots.index_order_02)
plots.plotly_matriz_movilidad(matriz)

In [7]:
df_tasas = plots.preparar_tasas_migracion_recepcion(data['set_abcde'], plots.orden_regiones)
plots.plotly_tasas_migracion_recepcion(df_tasas)

In [8]:
# Paso 1: Calcular tasas
df_tasas = plots.calcular_tasas_migracion(data['set_abcde'])

# Paso 2: Calcular distancias con Haversine
df_con_distancias = plots.calcular_distancias(data['set_abcde'])

# Paso 3: Calcular distancias promedio por región
df_resultado = plots.calcular_distancias_promedio(df_con_distancias, df_tasas)

# Paso 4: Visualización interactiva
plots.plotly_migracion_vs_distancia(df_resultado)

In [9]:
# O también, una variante:
plots.plotly_tasa_vs_distancia(
    df_resultado,
    col_x='DISTANCIA_PROMEDIO_RECEPCIÓN',
    col_y='Tasa Recepción (%)',
    titulo='Tasa de Recepción vs Distancia Promedio de Recepción',
    color='orange'
)

In [10]:
# ✅ Código para guardar los DataFrame

# Crear carpeta si no existe
output_dir = '../data/plots'
os.makedirs(output_dir, exist_ok=True)

# 1. Tipo universidad por región
df_tipo_univ = plots.preparar_dataset_tipo_universidad(data['set_a'], plots.tipos_universidad, plots.orden_regiones)
df_tipo_univ.to_csv(f'{output_dir}/df_tipo_universidad.csv', index=True)

# 2. Tipo de dependencia por región
df_tipodepen = plots.preparar_dataset_tipodepen(data['set_c'], plots.regiones_dict, plots.tipodepen_dict, plots.index_order)
df_tipodepen.to_csv(f'{output_dir}/df_tipodepen.csv', index=True)

# 3. IVM por región
df_ivm = plots.preparar_dataset_ivm(data['set_abcde'], plots.regiones_dict, plots.index_order)
df_ivm.to_csv(f'{output_dir}/df_ivm.csv', index=True)

# 4. Matriz de movilidad (de origen a destino)
matriz_mov = plots.preparar_matriz_movilidad(data['set_ab'], plots.index_order_02)
matriz_mov.to_csv(f'{output_dir}/df_matriz_movilidad.csv', index=True)

# 5. Tasas de migración y recepción
df_tasas_migracion = plots.preparar_tasas_migracion_recepcion(data['set_abcde'], plots.orden_regiones)
df_tasas_migracion.to_csv(f'{output_dir}/df_tasas_migracion.csv', index=False)

# 6. Distancia promedio vs tasas
df_tasas = plots.calcular_tasas_migracion(data['set_abcde'])
df_con_dist = plots.calcular_distancias(data['set_abcde'])
df_resultado = plots.calcular_distancias_promedio(df_con_dist, df_tasas)
df_resultado.to_csv(f'{output_dir}/df_migracion_distancia.csv', index=False)

In [11]:
# ✅ Código para guardar una muestra de cada conjunto

# Año y rutas
year = 2021
DATA_DIR = Path(f"../data/clean/{year}")
OUTPUT_DIR = Path("../data/plots")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Leer todos los archivos set_*.csv
archivos = list(DATA_DIR.glob('set_*.csv'))

# Guardar muestras (por ejemplo, las primeras 100 filas)
for archivo in archivos:
    nombre = archivo.stem  # nombre del archivo sin .csv
    df = pd.read_csv(archivo)
    df.head(100).to_csv(OUTPUT_DIR / f"{nombre}_sample.csv", index=False)


In [16]:
df_tipo_univ

tipo_inst_3,Universidades Estatales CRUCH,Universidades Privadas,Universidades Privadas CRUCH
region_sede,,,
Arica y Parinacota,40.000000,60.000000,0.000000
Tarapacá,40.000000,60.000000,0.000000
Antofagasta,22.222222,66.666667,11.111111
Atacama,33.333333,66.666667,0.000000
Coquimbo,11.111111,77.777778,11.111111
Valparaíso,18.181818,63.636364,18.181818
Metropolitana,21.052632,63.157895,15.789474
Lib. Gral B. O'Higgins,40.000000,60.000000,0.000000
Maule,14.285714,71.428571,14.285714


In [15]:
df_tipo = pd.read_csv("../data/plots/df_tipo_universidad.csv")
plots.plotly_tipo_universidad_por_region(df_tipo)